# Learning to Learn w\ Quantum NN via Classical NN

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfnmangini/QOSF-Mentorship/blob/main/l2l_0.ipynb)

This it the first jupyter notebook used to familiarize with _Pennylane_ and _RNNs_ and _LSTMs_

**Run cell below only if using this notebook in Colab**  
Installing required dependencies and packages

In [ ]:
in_Colab = False
if in_Colab:
    !pip install --upgrade qiskit
    !pip install --upgrade pennylane

#### Importation of required packages

In [3]:
import pennylane as qml
import qiskit as qk
import tensorflow as tf
import numpy as np

This is a test.

In [4]:
a = np.random.rand(3)
a

array([0.68358837, 0.04682249, 0.06980107])